In [20]:
import pandas as pd

ex_data = pd.read_excel('data_mail.xlsx', sheet_name="deti.mail")

data = pd.DataFrame(ex_data, columns=['URL', 'Title', 'Description', 'Article Text', 'Keywords'])

In [21]:
data['URL'][0]

'https://deti.mail.ru/article/kostyumy-na-novyj-god-dlya-detej-svoimi-rukami/'

In [22]:
import requests
from bs4 import BeautifulSoup

articles, titles, descriptions = [], [], []
for i in range(20):
    url=data['URL'][i]
    #url='https://sportmail.ru/news/football-eurocups/60231388/'
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f'Error:{resp.status_code}. Adress:{url}')

    soup = BeautifulSoup(resp.text, 'lxml')
    try:
        block = soup.find('header')
        block1 = soup.find('main').find_all('div', {'article-item-type': 'html'})

        title = block.h1.text
        descr = block.find_all('div')[1].text

    except:
        title = soup.h1.text
        try:
            descr = soup.find('div', {'class': 'article__intro'}).text
            block1 = soup.find_all('div', {'class': 'article__item article__item_alignment_left article__item_html'})
        except:
            descr = soup.find('div', {'class': 'page-info__lead'}).text
            block1 = soup.find_all('div', {'class': 'article__item article__item_alignment_left article__item_text article__item_html'})

    article = ''
    for k in block1:
        j = k.text
        if 'Читайте также' in j:
            break
        article += j + '\n'
    if len(article) < 20:
        print('Problem with article')
        print(i)
    articles.append(article)
    titles.append(title)
    descriptions.append(descr)

In [23]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI

base_url = "http://mistral.vkcloud.eazify.net:8000/v1"

chat = ChatOpenAI(api_key="<key>",
                model = "tgi",
                openai_api_base = base_url,
                temperature=0.2)

instruct = """

Прочитай приведённый ниже в тройных обратных кавычках статью и выведи из нее пять тегов (ключевых слов) в именительном падеже
в формате JSON следующего вида:
{{
  "Tags": ["", "", "", "", ""]
}}
Статья: ```{review}```
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты аналитик сайта статей, и твоя задача извлекать из статей теги (ключевые слова)."),
        ("human", instruct),
    ]
)


In [24]:
from langchain_core.output_parsers import JsonOutputParser
from tqdm.auto import tqdm

parser = JsonOutputParser()
res = []

for x in articles:
    z = chat.invoke(chat_template.format_messages(review=x[:900]))
    try:
        res.append(parser.invoke(z))
    except:
        pass

In [25]:
base_url = "http://mistral.vkcloud.eazify.net:8000/v1"

chat_2 = ChatOpenAI(api_key="<key>",
                model = "tgi",
                openai_api_base = base_url,
                temperature=0.2)

instruct_2 = """

Прочитай приведённый ниже в тройных обратных кавычках статью и определи к какому классу из списка classes = ["Политика", "Экономика", "Детство",
"Общество", "Кино" , "Телевидение", "Персоны", "События", "Бредны", "Наука", "Гаджеты", "Соцсети", "Технологии", "Опросы", "Головоломки", "Дом", "Транспорт", "Погода", "Мода", "Красота"] относится 
эта статья и запиши ответ в формате JSON следующего вида:
{{
  "Class": ""
}}
Статья: ```{review}```"
"""

chat_template_class = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты аналитик сайта статей, и твоя задача определить класс статьи, основываясь на текст статьи."),
        ("human", instruct_2),
    ]
)



In [26]:
from langchain_core.output_parsers import JsonOutputParser
from tqdm.auto import tqdm

parser = JsonOutputParser()
cl = []

for x in articles:
    z = chat.invoke(chat_template_class.format_messages(review=x[:900]))
    try:
        cl.append(parser.invoke(z))
    except:
        pass

In [27]:
clas, resul = [], []
for i in res:
    resul.append(i['Tags'])
for i in cl:
    clas.append(i['Class'])
data = {"Title": titles,
        "Description": descriptions,
        "Article Text": articles,
        "Tags": resul,
        "Class": clas
}

In [28]:
df = pd.DataFrame(data)
df

,Title,Description,Article Text,Tags,Class
0,Костюмы на Новый год для детей своими руками,Сделать новогодний наряд для сына или дочки св...,Костюм Снежинки\nВ качестве основы для костюма...,"{'Tags': ['Снежинки', 'юбка', 'платье', 'голов...",{'Class': 'Детство'}
1,Как создают детское новогоднее шоу: интервью с...,"Режиссер, сценарист и главный новогодний волше...",Если ребенок верит в сказку хотя бы до десяти ...,"{'Tags': ['сказка', 'ребенок', 'Дед Мороз', 'т...",{'Class': 'Детство'}
2,30 крутых детских новогодних подарков на любой...,"Думаете о том, что подарить ребенку на Новый г...",Подарки до 1 000 рублей\nНаборы для создания ...,"{'Tags': ['Подарки', 'Снежный шар', 'Рождестве...",{'Class': 'Детство'}
3,"Что делать, если с ГВ не сложилось: 4 совета","Лучшая еда для младенца – это, конечно, мамино...",Чаще практикуйте контакт кожа к коже\nПериод ...,"{'Tags': ['кожа к коже', 'контакт', 'новорожде...",{'Class': 'Детство'}
4,Новый год близко: как звезды с детьми наряжают...,Главный атрибут праздника уже появился в домах...,"На календаре почти середина декабря, и знамени...","{'Tags': ['Новый год', 'Сергей Лазарев', 'Дед ...",{'Class': 'Детство'}
5,Дорогой Дедушка Мороз: подборка смешных детски...,Собрали самые веселые и трогательные письма Де...,Приятная тенденция этого года – многие дети пи...,"{'Tags': ['ребенок', 'Дед Мороз', 'Новый год',...",{'Class': 'Детство'}
6,Послеродовая депрессия у папы: как распознать ...,Если о материнской послеродовой депрессии в по...,В мае 2010 года в научном журнале Journal of t...,"{'Tags': ['депрессия', 'беременность', 'рожден...",{'Class': 'Наука'}
7,Как красиво одеть ребенка на Новый год и не ра...,"Новогодних утренников, елок и прочих культпохо...",На ёлку\nГлавное новогоднее событие для детеи...,"{'Tags': ['Елка', 'Дед Мороз', 'Снегурочка', '...",{'Class': 'Детство'}
8,Как украсить детскую комнату к Новому году: ид...,Оказываясь в отделе магазина с новогодними укр...,Рисуем\nУкрасить окно обычными снежинками и во...,"{'Tags': ['кисточка', 'акриловые краски', 'мар...",{'Class': 'Детство'}
9,"5 главных опасностей, которые угрожают ребенку...",Какие домашние травмы – самые распространенные...,"Рано или поздно дети получают первые ушибы, ца...","{'Tags': ['ожоги', 'царапины', 'синяки', 'дома...",{'Class': 'Детство'}


In [ ]:
df.to_json('datas.json')